# Web Scraping

# Main program to scrape www.metroscubicos.com

In [ ]:
import requests
import bs4
import json
url = "https://www.inmuebles24.com/departamentos-en-venta-en-del-carmen.html"
root_url = "https://www.inmuebles24.com"
page = requests.get(url)

In [ ]:
import requests
import bs4
import json

class WebScraping(object):
    
    def __init__(self):
        super().__init__()
        
        json_file_name = 'ws_parameters.json'
        source_to_test = 'metroscubicos'

        # open json file to get page ID and token
        with open(json_file_name, 'r') as f:
            data = json.load(f)

        self.main_url = data['app']['sources'][source_to_test]['main_url']
        self.output_file_name = data['app']['sources'][source_to_test]['output_file_name']
        self.debug = data['app']['sources'][source_to_test]['debug']
        self.delimiter = data['app']['sources'][source_to_test]['delimiter']
        
        if self.debug == 'Y':
            print('In WebScraping')
            print(self.main_url)
            
    def write_file(self, p_line):
        self.output_file.write(p_line + "\n")

    def next_page(self, p_soup):
        new_url = 'None'
        pag_container = p_soup.find_all("a", class_="andes-pagination__link prefetch")
        if self.debug == 'Y':
            print('Type: ' + str(type(pag_container)) + ' with ' + str(len(pag_container)) + ' elements')
        for item in pag_container:
            new_url = item.get('href')
        if self.debug == 'Y':
            print(new_url)
        return new_url
    
    def get_url(self, p_data_links, p_id, p_initial_id):
# There is no need to loop the entire dataset, p_initial_id parameter helped to solve this
#        item_url = ''
#        for j, itemj in enumerate(p_data_links):
#            if j + len(p_data_links) == p_id:
#                item_url = itemj['href']
#                break
        item_url = p_data_links[p_id-p_initial_id]['href']
        return item_url

    def read_website(self, p_url):
        if self.debug == 'Y':
            print('In read_website')
        
        url_to_search = p_url
        k = 0 # To manage the counter accross pages
        
        while url_to_search != 'None':
            print('Searching ' + url_to_search)
            page = requests.get(url_to_search)
            soup = bs4.BeautifulSoup(page.content, 'html.parser')
            data_block = soup.find_all("div", class_="item__info")
            data_links = soup.find_all("a", class_="item__info-link")
            if self.debug == 'Y':
                print('data_block type: ' + str(type(data_block)) + ' with ' + str(len(data_block)) + ' elements')
                print('data_links type: ' + str(type(data_links)) + ' with ' + str(len(data_links)) + ' elements')

            for i, item in enumerate(data_block):
                prices = item.find_all("span", class_="price__symbol")
                pricev = item.find_all("span", class_="price__fraction")
                title = item.find_all("div", class_="item__title")
                htype = item.find_all("p", class_="item__info-title")
                descr = item.find_all("div", class_="item__attrs")
                item_url = self.get_url(data_links, i+k, k)
                line = str(i+k+1) + self.delimiter + prices[0].contents[0] + self.delimiter + pricev[0].contents[0] + self.delimiter + title[0].contents[0] + self.delimiter + htype[0].contents[0] + self.delimiter + descr[0].contents[0] + self.delimiter + item_url
                self.write_file(line)

                if self.debug == 'Y':
                    print(f'{str(i+k+1)} {self.delimiter} {prices[0].contents[0]} {self.delimiter} {pricev[0].contents[0]} {self.delimiter} {title[0].contents[0]} {self.delimiter} {htype[0].contents[0]} {self.delimiter} {descr[0].contents[0]} {self.delimiter} {item_url}')

            url_to_search = self.next_page(soup)
            k = i + k + 1

def main():
    print('Start')
    app = WebScraping()
    app.output_file = open(app.output_file_name,"w+")
    app.read_website(app.main_url)
    app.output_file.close()
    print('End')

if __name__ == '__main__':
    main()

# Main program to scrape www.inmuebles24.com

## Testing using only the start page

In [126]:
import requests
import bs4
import json

class WebScraping(object):
    
    def __init__(self):
        super().__init__()
        
        json_file_name = 'ws_parameters.json'
        source_to_test = 'inmuebles24'

        # open json file to get page ID and token
        with open(json_file_name, 'r') as f:
            data = json.load(f)

        self.main_url = data['app']['sources'][source_to_test]['main_url']
        self.root_url = data['app']['sources'][source_to_test]['root_url']
        self.output_file_name = data['app']['sources'][source_to_test]['output_file_name']
        self.debug = data['app']['sources'][source_to_test]['debug']
        self.delimiter = data['app']['sources'][source_to_test]['delimiter']
        
        if self.debug == 'Y':
            print('In WebScraping')
            print(self.main_url)
            
    def write_file(self, p_line):
        self.output_file.write(p_line + "\n")

    def next_page(self, p_soup):
        new_url = 'None'
        pag_container = p_soup.find_all("li", class_="pag-go-next")
        if self.debug == 'Y':
            print('Type: ' + str(type(pag_container)) + ' with ' + str(len(pag_container)) + ' elements')
        for item in pag_container:
            new_url = self.root_url + item.findChild("a")['href']
        return new_url
    
    def get_url(self, p_data_links, p_id, p_initial_id):
        item_url = p_data_links[p_id-p_initial_id]['href']
        if self.debug == 'Y':
            print(item_url)
        return item_url

    def read_website(self, p_url):
        if self.debug == 'Y':
            print('In read_website')
        
        url_to_search = p_url
        k = 0 # To manage the counter accross pages
        prices_list = list()
        line = "Id" + self.delimiter + "URL" + self.delimiter + "Descripción" + self.delimiter + "Precio" + self.delimiter + \
                "Terreno" + self.delimiter + "Construcción" + self.delimiter + "Recámaras" + self.delimiter + \
            "Baños" + self.delimiter + "Estacionamiento"
        self.write_file(line)
        
        while url_to_search != 'None':
            print('Searching ' + url_to_search)
            page = requests.get(url_to_search)
            soup = bs4.BeautifulSoup(page.content, 'html.parser')
            data_block = soup.find_all("div", class_="posting-info")
            data_links = soup.find_all("a", class_="go-to-posting")
            data_prices = soup.find_all("span", class_="first-price")
            
            if self.debug == 'Y':
                print('data_block type: ' + str(type(data_block)) + ' with ' + str(len(data_block)) + ' elements')
                print('data_links type: ' + str(type(data_links)) + ' with ' + str(len(data_links)) + ' elements')
                print('data_prices type: ' + str(type(data_prices)) + ' with ' + str(len(data_prices)) + ' elements')

            for i, item in enumerate(data_prices):
                item_price = item.text
                prices_list.append(item_price)
                
            for i, item in enumerate(data_block):
                item_anchor = item.find_all("a", class_="go-to-posting")
                item_features_ul = item.find_all("ul", class_="main-features go-to-posting")
                item_description_div = item.find_all("div", class_="posting-description go-to-posting")
                item_url = self.get_url(data_links, i+k, k)
                line = str(i+k+1) + self.delimiter + self.root_url + item_anchor[0]["href"] + self.delimiter + \
                        item_description_div[0].text.strip().replace("\t"," ") + self.delimiter + \
                        prices_list[i] + self.delimiter + item_features_ul[0].text.strip().replace("\n",delimiter)
                self.write_file(line)

                if self.debug == 'Y':
                    print(f'{str(i+k+1)} {self.delimiter} {self.root_url + item_anchor[0]["href"] } {self.delimiter} {item_description_div[0].text.strip()} {self.delimiter} {item_features_ul[0].text} {self.delimiter}')

            url_to_search = self.next_page(soup)
            k = i + k + 1

def main():
    print('Start')
    app = WebScraping()
    app.output_file = open(app.output_file_name,"w+")
    app.read_website(app.main_url)
    app.output_file.close()
    print('End')

if __name__ == '__main__':
    main()

Start
Searching https://www.inmuebles24.com/inmuebles-en-jardin-balbuena.html
Searching https://www.inmuebles24.com/inmuebles-en-jardin-balbuena-pagina-2.html
Searching https://www.inmuebles24.com/inmuebles-en-jardin-balbuena-pagina-3.html
Searching https://www.inmuebles24.com/inmuebles-en-jardin-balbuena-pagina-4.html
Searching https://www.inmuebles24.com/inmuebles-en-jardin-balbuena-pagina-5.html
Searching https://www.inmuebles24.com/inmuebles-en-jardin-balbuena-pagina-6.html
Searching https://www.inmuebles24.com/inmuebles-en-jardin-balbuena-pagina-7.html
Searching https://www.inmuebles24.com/inmuebles-en-jardin-balbuena-pagina-8.html
Searching https://www.inmuebles24.com/inmuebles-en-jardin-balbuena-pagina-9.html
Searching https://www.inmuebles24.com/inmuebles-en-jardin-balbuena-pagina-10.html
Searching https://www.inmuebles24.com/inmuebles-en-jardin-balbuena-pagina-11.html
Searching https://www.inmuebles24.com/inmuebles-en-jardin-balbuena-pagina-12.html
Searching https://www.inmueb

In [ ]:
f = open("test.txt","w+")
soup = bs4.BeautifulSoup(page.content, 'html.parser')
data_block = soup.find_all("div", class_="posting-info")
print('Type: ' + str(type(data_block)) + ' with ' + str(len(data_block)) + ' elements')
#print(soup.prettify())
#print(str(data_block))
f.write(str(data_block))
#data_links = soup.find_all("a", class_="item__info-link")
f.close()

In [ ]:
i = 1
delimiter = '\t'
print("Starting loop")
for item in data_block:
#    print(str(i) + ' Type: ' + str(type(item)) + ' Name: ' + item.name)
#    print(item['class'])
    item_anchor = item.find_all("a", class_="go-to-posting")
    item_url = root_url + item_anchor[0]["href"]
    item_features_ul = item.find_all("ul", class_="main-features go-to-posting")
    item_features = item_features_ul[0].text.replace("\n",delimiter)
    item_description_div = item.find_all("div", class_="posting-description go-to-posting")
    item_description = item_description_div[0].text.strip()
    print('*******')
    print(item_features.strip())
    print('********************')
    print(item_description)
    print('*******')
    print(item_url)
#    print('Value: ' + str(item))
    i = i + 1

## Searching for Del Carmen Coyoacán

In [ ]:
url = "https://listado.metroscubicos.com/santa-luc%C3%ADa-reacomodo#D[A:santa-luc%C3%ADa-reacomodo]"
page = requests.get(url)
soup = bs4.BeautifulSoup(page.content, 'html.parser')
data_block = soup.find_all("div", class_="item__info")
print('Type: ' + str(type(data_block)) + ' with ' + str(len(data_block)) + ' elements')

In [ ]:
data_block

In [ ]:
xdelimiter = '\t'
for i, item in enumerate(data_block):
    prices = item.find_all("span", class_="price__symbol")
    pricev = item.find_all("span", class_="price__fraction")
    title = item.find_all("div", class_="item__title")
    htype = item.find_all("p", class_="item__info-title")
    descr = item.find_all("div", class_="item__attrs")
    print(item)
#    print(f'{str(i+1)} {self.delimiter} {prices[0].contents[0]} {self.delimiter} {pricev[0].contents[0]} {self.delimiter} {title[0].contents[0]} {self.delimiter} {htype[0].contents[0]} {self.delimiter} {descr[0].contents[0]}')

In [ ]:
data_links = soup.find_all("a", class_="item__info-link")
print('Type: ' + str(type(data_links)) + ' with ' + str(len(data_links)) + ' elements')
for i, item in enumerate(data_links):
    #descr = item.find_all("div", class_="item__attrs")
    print(item['href'])
#    print(f'{str(i+1)} {self.delimiter} {prices[0].contents[0]} {self.delimiter} {pricev[0].contents[0]} {self.delimiter} {title[0].contents[0]} {self.delimiter} {htype[0].contents[0]} {self.delimiter} {descr[0].contents[0]}')

In [ ]:
pag_container = soup.find_all("a", class_="andes-pagination__link prefetch")
print('Type: ' + str(type(pag_container)) + ' with ' + str(len(pag_container)) + ' elements')
for item in pag_container:
    nurl = item.get('href')
    print(nurl)

In [ ]:
!pip install beautifulsoup4
import requests
import bs4